# Decrypting an RSA-Encrypted Message

In this project, we want to decrypt an encoded message Alice sent to Bob. To do this, we need to decrypt each number in the list using Bob’s private key. The RSA decryption formula is:

$$ m = c^d \mod n $$

where:

- $ c $ is the ciphertext (each large encrypted number).
- $ d $ is the private exponent, which is computed as:
  $$ d = e^{-1} \mod \varphi(n) $$
  where $ \varphi(n) $ is Euler’s totient function of $ n $.
- $ n $ is the modulus.

### Steps to Solve

#### 1. Compute Bob's Private Key (Exponent $ d $)

1. Factorize $ n $ into its two prime factors: $ p $ and $q$.

2. Compute Euler’s totient function:
   $$ \varphi(n) = (p - 1)(q - 1) $$

3. Compute $ d $, the modular inverse of $ e $ modulo $ \varphi(n)$.

#### 2. Decrypt Each Ciphertext Value

- Compute:
  $$ m = c^d \mod n $$
  for each encrypted number in the list.

#### 3. Decode the Resulting Numbers

- Convert each number $ m $ into letters using the agreed encoding scheme.

### Expected Outcome

After decrypting, we will get a number sequence that maps to letters or spaces according to the agreed encoding scheme. This will reveal the original message Alice sent to Bob.

### Computing Bob's Private Key, $d$

We will first define the RSA modulus $n$, the public exponent $e$, and the message to be encrypted as a list of ciphertext values $M$. In this case we call the public key the tuple $(e,n)$

In [39]:
# Given information
n = 956331992007843552652604425031376690367
e = 12398737
M = [427849968240759007228494978639775081809,
     498308250136673589542748543030806629941,
     925288105342943743271024837479707225255,
     95024328800414254907217356783906225740]

Next we  will factor the RSA modulus $n$ into its two distinct prime factors. We use Sympy's `factorint` to get the factors, check that \(n\) is a semiprime (exactly two primes, each with exponent 1), and then return the two primes $p$ and $q$ (sorted in increasing order).

In [42]:
# Factoring n into two primes, p and q
from sympy import factorint

def factorize_prime(n):
    """Factorizes n into its two prime factors using sympy's factorint."""
    factors = factorint(n)
    # Expecting exactly two prime factors with exponent 1 (i.e. semiprime)
    if len(factors) != 2 or not all(exp == 1 for exp in factors.values()):
        raise ValueError("n is not a semiprime with two distinct prime factors")
    # Extract the factors
    p, q = sorted(factors.keys())
    return p, q

p, q = factorize_prime(n)
print("p =", p)
print("q =", q)

p = 7746289204980135457
q = 123456789012345681631


Next we compute the Euler’s totient function:
   $$ \varphi(n) = (p - 1)(q - 1) $$

In [44]:
# Compute Euler's totient function φ(n)
phi_n = (p - 1) * (q - 1)
print(f"Euler's Totient Function φ({n}) = {phi_n}")

Euler's Totient Function φ(956331992007843552652604425031376690367) = 956331992007843552521401346814050873280


Next, we define a function that implements the extended Euclidean algorithm. In summary, it computes the greatest common divisor (gcd) of two integers $a$ and $b$ and finds integers $u$ and $v$ such that:

$$a\cdot u + b\cdot v = \gcd(a,b)$$

It does this by iteratively updating remainders and corresponding coefficients until the remainder reaches zero.

In [46]:
#  Extended Euclidean Algorithm
def Extended_Euclidean_Algorithm(a, b):
    if a == 0 or b == 0:
        return "Error"

    r0, r1 = a, b
    s0, s1 = 1, 0
    t0, t1 = 0, 1

    while r1 > 0:
        q = r0 // r1
        r = r0 - q * r1
        r0, r1 = r1, r

        s = s0 - q * s1
        s0, s1 = s1, s

        t = t0 - q * t1
        t0, t1 = t1, t

    gcd = r0
    u = s0
    v = t0
    return gcd, u, v

We use the Extended Euclidean Algorithm to compute the modular inverse of $a$ modulo $n$. Particularly, our function calculates Bob's private key $d$ as the inverse of the public exponent $e$ modulo $\phi(n)$, ensuring that $e \times d \equiv 1 \pmod{\phi(n)}$. The result is then printed as Bob's private key.

In [48]:
# Function for inverse mod n
def inverse_mod_n(a, n):
    gcd, u, v = Extended_Euclidean_Algorithm(a, n)
    if gcd == 1:
        return u % n
    else:
        return "The inverse does not exist."  # a and n are not coprime

# Compute Bob's private key (d)
d = inverse_mod_n(e, phi_n)
print(f"Bob's private key (d): {d}")

Bob's private key (d): 801756262003467870842260800571951669873


### Decrypting Each Ciphertext Value i.e, Discovering the Message Value/Numbers

Next, we iterate over each ciphertext in $M$ and apply the RSA decryption formula $m = c^d \mod n$. The decrypted numeric values (representing the original encoded message) are collected in the list `decrypted_message` and then printed.

In [50]:
# Decrypt each ciphertext value
decrypted_message = []
for c in M:
    m = pow(c, d, n)
    decrypted_message.append(m)
print(f"Decrypted message: {decrypted_message}")


Decrypted message: [30181929001929002335002215303015280030, 25003018150033252822140030181130002415, 32152800332825301500302500231500152319, 223500141913211924292524]


### Decoding the Message Value

We will define a dictionary (`decode_map`) that maps two-digit codes (like '$11$' for $A$, '$12$' for $B$, etc., and '$00$' for a space) to their corresponding characters. We then concatenate the decrypted integers into one string, and ensure the string has an even length by prepending a "$0$" if needed.

In [58]:
 # Mapping dictionary for the encoding scheme
decode_map = {
        '00': ' ',
        '11': 'A', '12': 'B', '13': 'C', '14': 'D', '15': 'E',
        '16': 'F', '17': 'G', '18': 'H', '19': 'I', '20': 'J',
        '21': 'K', '22': 'L', '23': 'M', '24': 'N', '25': 'O',
        '26': 'P', '27': 'Q', '28': 'R', '29': 'S', '30': 'T',
        '31': 'U', '32': 'V', '33': 'W', '34': 'X', '35': 'Y',
        '36': 'Z'
}


 # Convert the integer to a two-digit string
num_str = "".join(str(m) for m in decrypted_message)

# Check length and add a 0 if necessary to ensure even length
if len(num_str) % 2 != 0:
    num_str = "0" + num_str

Finally, we split the concatenated numeric string into two-digit groups, translate each pair into its corresponding character using `decode_map`, and prints the resulting hidden message.

In [63]:
# Finally decoding the message using the mapping
hidden_message = ""
for i in range(0, len(num_str), 2):
    pair = num_str[i:i+2]
    if pair in decode_map:
        hidden_message += decode_map[pair]
    else:
        hidden_message += "?" # Handle cases where the pair is not in the mapping

print("Hidden message:", hidden_message)

Hidden message: THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON


The final decoded message is:  
**THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON**

This message from to the famous poem by Emily Dickinson.